This file is for testing sar_dataset_loader.py

TODOS (All done):
1. ~~Read from real files or metadata dataframe~~
2. ~~Depth (height) adjust the wind speed~~
3. ~~Make sure the logic works in situations with both wave height parameters~~
4. ~~Make sure it works with files with both wave height and wind speed in the survey data~~

In [1]:
import sys
sys.path.insert(1, '/home/sarssw/filip/sarssw')

In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import os
import pickle
from tqdm import tqdm
import math
import pipeline.sar_dataset_loader as sar_dataset_loader
from importlib import reload

In [3]:
#Load bouy survey dataframe
bouy_survey_fn = '../bouy_survey/1h_survey/result_df'
swh_model_fn = '/data/exjobb/sarssw/model/2021_swh_era5_world_wide.nc'
wspd_model_fn = '/data/exjobb/sarssw/model/WIND_GLO_PHY_global/all.nc'

In [4]:
sar_dir = '/data/exjobb/sarssw/sar_dataset/'   
svc_file = '/home/sarssw/filip/sarssw/pipeline/homogenity_svc.pkl'
all_sar_images = os.listdir(sar_dir)

In [5]:
sar_paths = [os.path.join(sar_dir, f) for f in all_sar_images[:10]]

In [6]:
bouy_survey_path = '../bouy_survey/1h_survey/result_df'

with open(bouy_survey_path,'rb') as f_r:
    bouy_survey_df = pickle.load(f_r)

### 3. Make sure the logic works in situations with both wave height parameters

In [7]:
url = 'https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_IW_GRDH_1SDV_20210409T175650_20210409T175715_037377_0467A5_96FF.zip'
bouy = 'GL_TS_MO_6200078.nc'
sub_df = bouy_survey_df[(bouy_survey_df['bouy_file_name'] == bouy) & (bouy_survey_df['sar_url'] == url)]
display(sub_df)

,bouy_file_name,bouy_longitude,bouy_latitude,bouy_time,bouy_depth_index,bouy_depth,bouy_variable_name,bouy_variable_value,sar_url,sar_mode,sar_polarization,sar_platform,sar_start_time,sar_stop_time,sar_coordinates
38009,GL_TS_MO_6200078.nc,-2.787,47.238998,2021-04-09 18:00:00,0,0.0,VAVH,0.299,https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_...,IW,"[VV, VH]",Sentinel-1A,2021-04-09 17:56:50,2021-04-09 17:57:15,"[[-3.24896, 48.195469], [-2.809912, 46.698505]..."
38238,GL_TS_MO_6200078.nc,-2.787,47.238998,2021-04-09 17:55:00,0,0.0,VHM0,0.399,https://datapool.asf.alaska.edu/GRD_HD/SA/S1A_...,IW,"[VV, VH]",Sentinel-1A,2021-04-09 17:56:50,2021-04-09 17:57:15,"[[-3.24896, 48.195469], [-2.809912, 46.698505]..."


In [8]:
sar_name = 'S1A_IW_GRDH_1SDV_20210409T175650_20210409T175715_037377_0467A5_96FF'
reload(sar_dataset_loader)
sar_paths = [os.path.join(sar_dir, f) for f in all_sar_images if sar_name in f]
display(sar_paths)
#features = sar_dataset_loader.load_features_df(sar_paths, svc_file)
#display(features)
dataset_df = sar_dataset_loader.load_features_labels_df(sar_paths, svc_file, bouy_survey_fn, swh_model_fn, wspd_model_fn)
display(dataset_df)

['/data/exjobb/sarssw/sar_dataset/S1A_IW_GRDH_1SDV_20210409T175650_20210409T175715_037377_0467A5_96FF-GL_TS_MO_6200078-2.nc',
 '/data/exjobb/sarssw/sar_dataset/S1A_IW_GRDH_1SDV_20210409T175650_20210409T175715_037377_0467A5_96FF-GL_TS_MO_6200078-5.nc',
 '/data/exjobb/sarssw/sar_dataset/S1A_IW_GRDH_1SDV_20210409T175650_20210409T175715_037377_0467A5_96FF-GL_TS_MO_6200078-0.nc',
 '/data/exjobb/sarssw/sar_dataset/S1A_IW_GRDH_1SDV_20210409T175650_20210409T175715_037377_0467A5_96FF-GL_TS_MO_6200078-8.nc',
 '/data/exjobb/sarssw/sar_dataset/S1A_IW_GRDH_1SDV_20210409T175650_20210409T175715_037377_0467A5_96FF-GL_TS_MO_6200078-3.nc',
 '/data/exjobb/sarssw/sar_dataset/S1A_IW_GRDH_1SDV_20210409T175650_20210409T175715_037377_0467A5_96FF-GL_TS_MO_6200078-4.nc',
 '/data/exjobb/sarssw/sar_dataset/S1A_IW_GRDH_1SDV_20210409T175650_20210409T175715_037377_0467A5_96FF-GL_TS_MO_6200078-6.nc',
 '/data/exjobb/sarssw/sar_dataset/S1A_IW_GRDH_1SDV_20210409T175650_20210409T175715_037377_0467A5_96FF-GL_TS_MO_6200078

Calculating features


100%|██████████| 9/9 [00:02<00:00,  4.09it/s]


Loading wave height model
Loading wind speed model


100%|██████████| 1/1 [00:00<00:00, 21.65it/s]


file_name  \
sar_name                                           bouy_name                                                             
S1A_IW_GRDH_1SDV_20210409T175650_20210409T17571... GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   
                                                   GL_TS_MO_6200078  S1A_IW_GRDH_1SDV_20210409T175650_20210409T1757...   

                                                                    pol  \
sar_name                                           bouy_name              
S1A_IW_GRDH_1SDV_20210409T175650_20210409T17571... GL_TS_MO_6200078  VV   
                                                   GL_TS_MO_6200078  VH   
                                                   GL_TS_MO_6200078  VV   
                                                   GL_TS_MO_6200078  VH   
                                                   GL_TS_MO_6200078  VV   
                                                   GL_TS_MO_6200078  VH   
                                                   GL_TS_MO_6200078  VV   
                                                   GL_TS_MO_6200078  VH   
                                                   GL_TS_MO_6200078  VV   
                                                   GL_TS_MO_6200078  VH   
                                                   GL_TS_MO_6200078  VV   
                                                   GL_TS_MO_6200078  VH   
                                                   GL_TS_MO_6200078  VV   
                                                   GL_TS_MO_6200078  VH   
                                                   GL_TS_MO_6200078  VV   
                                                   GL_TS_MO_6200078  VH   
                                                   GL_TS_MO_6200078  VV   
                                                   GL_TS_MO_6200078  VH   

                                                                                                                  safe  \
sar_name                                     

## 4. Make sure it works with files with both wave height and wind speed in the survey data


In [9]:
v_count = bouy_survey_df.value_counts(subset=['bouy_file_name', 'sar_url'])
print(v_count.index[1])

reload(sar_dataset_loader)
sar_name = 'S1B_IW_GRDH_1SDV_20210104T064133_20210104T064158_025001_02F9B9_1DF4'

sar_paths = [os.path.join(sar_dir, f) for f in all_sar_images if sar_name in f]
#display(sar_paths)

dataset_df = sar_dataset_loader.load_features_labels_df(sar_paths, svc_file, bouy_survey_fn, swh_model_fn, wspd_model_fn)
display(dataset_df)

('IR_TS_MO_6201070.nc', 'https://datapool.asf.alaska.edu/GRD_HD/SB/S1B_IW_GRDH_1SDV_20210104T064133_20210104T064158_025001_02F9B9_1DF4.zip')
Calculating features


100%|██████████| 42/42 [00:00<00:00, 71.19it/s]


Loading wave height model
Loading wind speed model


100%|██████████| 5/5 [00:00<00:00, 66.65it/s]


file_name  \
sar_name                                           bouy_name                                                             
S1B_IW_GRDH_1SDV_20210104T064133_20210104T06415... IR_TS_MO_6200083  S1B_IW_GRDH_1SDV_20210104T064133_20210104T0641...   
                                                   IR_TS_MO_6200083  S1B_IW_GRDH_1SDV_20210104T064133_20210104T0641...   
                                                   IR_TS_MO_6200083  S1B_IW_GRDH_1SDV_20210104T064133_20210104T0641...   
                                                   IR_TS_MO_6200083  S1B_IW_GRDH_1SDV_20210104T064133_20210104T0641...   
                                                   IR_TS_MO_6200083  S1B_IW_GRDH_1SDV_20210104T064133_20210104T0641...   
...                                                                                                                ...   
                                                   IR_TS_MO_6201070  S1B_IW_GRDH_1SDV_20210104T064133_20210104T0641...   
                                                   IR_TS_MO_6201070  S1B_IW_GRDH_1SDV_20210104T064133_20210104T0641...   
                                                   IR_TS_MO_6201070  S1B_IW_GRDH_1SDV_20210104T064133_20210104T0641...   
                                                   IR_TS_MO_6201070  S1B_IW_GRDH_1SDV_20210104T064133_20210104T0641...   
                                                   IR_TS_MO_6201070  S1B_IW_GRDH_1SDV_20210104T064133_20210104T0641...   

                                                                    pol  \
sar_name                                           bouy_name              
S1B_IW_GRDH_1SDV_20210104T064133_20210104T06415... IR_TS_MO_6200083  VV   
                                                   IR_TS_MO_6200083  VH   
                                                   IR_TS_MO_6200083  VV   
                                                   IR_TS_MO_6200083  VH   
                                                   IR_TS_MO_6200083  VV   
...                                                                  ..   
                                                   IR_TS_MO_6201070  VH   
                                                   IR_TS_MO_6201070  VV   
                                                   IR_TS_MO_6201070  VH   
                                                   IR_TS_MO_6201070  VV   
                                                   IR_TS_MO_6201070  VH   

                                                                                                                  safe  \
sar_name                                           bouy_name                                                             
S1B_IW_GRDH_1SDV_20210104T064133_20210104T06415... IR_TS_MO_6200083  S1B_IW_GRDH_1SDV_20210104T064133_20210104T0641...   
                                                   IR_TS_MO_6200083  S1B_IW_GRDH_1SDV_20210104T064133_20210104T0641...   
                                                   IR_TS_MO_6200083  S1B_IW_GRDH_1SDV_20210104T064133_20210104T0641...   
                                                   IR_TS_MO_6200083  S1B_IW_GRDH_1SDV_20210104T064133_20210104T0641...   
                                                   IR_TS_MO_6200083  S1B_IW_GRDH_1SDV_20210104T064133_20210104T0641...   
...                                                                                                                ...   
                                                   IR_TS_MO_6201070  S1B_IW_GRDH_1SDV_20210104T064133_20210104T0641...   
                                                   IR_TS_MO_6201070  S1B_IW_GRDH_1SDV_20210104T064133_20210104T0641...   
                                                   IR_TS_MO_6201070  S1B_IW_GRDH_1SDV_20210104T064133_20210104T0641...   
                                                   IR_TS_MO_6201070  S1B_IW_GRDH_1SDV_20210104T064133_20210104T0641...   
                                                   IR_TS_MO_6201070  S1B_IW_GRDH_1